In [ ]:
# Kaggle Dataset: https://www.kaggle.com/c/dogs-vs-cats/data

In [24]:
import os
import cv2

import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

In [13]:
os.makedirs("./data/train/dog", exist_ok=True)
os.makedirs("./data/train/cat", exist_ok=True)

base_path = "./data/train"

for image in os.listdir(base_path):
    if os.path.isfile(f"{base_path}/{image}"):
        if "cat" in image:
            os.rename(f"{base_path}/{image}", f"{base_path}/cat/{image}")
        else:
            os.rename(f"{base_path}/{image}", f"{base_path}/dog/{image}")

In [25]:
def pre_process_image(folder_path):
    for image in os.listdir(folder_path):
        img = cv2.imread(f"{folder_path}/{image}", cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (150, 150))
        cv2.imwrite(f"{folder_path}/{image}", img)

pre_process_image("./data/train/dog")
pre_process_image("./data/train/cat")

In [28]:
rotation = {
    "90_clockwise": cv2.ROTATE_90_CLOCKWISE,
    "180": cv2.ROTATE_180,
    "90_counter_clockwise": cv2.ROTATE_90_COUNTERCLOCKWISE
}

def rotate_image(folder_path):
    for image in os.listdir(folder_path):
        for rot in rotation.keys():
            img = cv2.imread(f"{folder_path}/{image}")
            img = cv2.rotate(img, rotation.get(rot))
            cv2.imwrite(f"{folder_path}/{rot}_{image}", img)

rotate_image("./data/train/dog")
rotate_image("./data/train/cat")

In [36]:
train_dataset, val_dataset = tf.keras.utils.image_dataset_from_directory(
    "./data/train",
    validation_split=0.2,
    label_mode="binary",
    batch_size=32,
    image_size=(150, 150),
    subset="both",
    seed=42,
)

Found 100000 files belonging to 2 classes.
Using 80000 files for training.
Using 20000 files for validation.


In [38]:
callback = tf.keras.callbacks.EarlyStopping(monitor="accuracy", patience=3)

In [39]:
model = models.Sequential([
    layers.Input(shape=(150, 150, 3)),
    
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    layers.MaxPool2D(pool_size=(2, 2)),
    
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPool2D(pool_size=(2, 2)),
    
    layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    layers.MaxPool2D(pool_size=(2, 2)),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [40]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 16)             │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,837,665 (18.45 MB)

 Trainable params: 4,837,665 (18.45 MB)

 Non-trainable params: 0 (0.00 B)

In [41]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [42]:
model.fit(
    train_dataset,
    batch_size=32,
    epochs=15,
    validation_data=val_dataset,
    callbacks=[callback]
)

Epoch 1/15
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 450s 179ms/step - accuracy: 0.5810 - loss: 1.2537 - val_accuracy: 0.6988 - val_loss: 0.5621
Epoch 2/15
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 486s 194ms/step - accuracy: 0.7484 - loss: 0.5162 - val_accuracy: 0.7916 - val_loss: 0.4479
Epoch 3/15
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 495s 198ms/step - accuracy: 0.7974 - loss: 0.4371 - val_accuracy: 0.7498 - val_loss: 0.5271
Epoch 4/15
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 457s 183ms/step - accuracy: 0.8409 - loss: 0.3579 - val_accuracy: 0.8001 - val_loss: 0.4454
Epoch 5/15
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 420s 168ms/step - accuracy: 0.8871 - loss: 0.2628 - val_accuracy: 0.7947 - val_loss: 0.5542
Epoch 6/15
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 433s 173ms/step - accuracy: 0.9232 - loss: 0.1902 - val_accuracy: 0.7801 - val_loss: 0.5641
Epoch 7/15
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 443s 177ms/step - accuracy: 0.9518 - loss: 0.1281 - val_accuracy: 0.7930 - val_loss: 0.6989
Epoch 8/15
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 418s 167ms/step - ac

In [43]:
model.evaluate(val_dataset)

625/625 ━━━━━━━━━━━━━━━━━━━━ 28s 45ms/step - accuracy: 0.8013 - loss: 1.5477


[1.5894896984100342, 0.7996000051498413]